In [1]:
import Base: iterate,max, exp, sin, cos, +, ^, -, *, /, sqrt, convert, promote_rule, zero
import Test: @test, @testset

In [2]:
2^2

4

# Define dual number

In [3]:
struct Infinitesimal{T <: Number} <: Number
    x::T
    dx::T
#    Infinitesimal{T}(x::Number, dx::Number) where {T <: Number}  = (print("dupa"); new(x, dx))
end

In [4]:
# struct Infinitesimal{T <: Number} <: Number
#     x::T
#     dx::T
#     Infinitesimal(x::T, dx::T) where {T <: Number}  = (print("dupa"); new(x, dx))
# end

In [5]:
Infinitesimal(1, 0)

Infinitesimal{Int64}(1, 0)

In [6]:
function convert(::Type{Infinitesimal{T}}, x::T) where {T}
   Infinitesimal(x, zero(x)) 
end
function convert(::Type{Infinitesimal{T}}, x::Infinitesimal{S}) where {S, T}
    Infinitesimal(T(x.x), T(x.dx))
end
# This is needed according to an error before
function convert(::Type{Infinitesimal{T}}, x::T) where {T <: Number}
    Infinitesimal(x, zero(x))
end
function convert(::Type{Infinitesimal{T}}, x::S) where {T, S <: Number}
    x_as_T = convert(T, x)
    Infinitesimal(x_as_T, zero(x_as_T))
end

convert (generic function with 187 methods)

In [7]:
h=4.0
z = Infinitesimal(1, zero(1))
h + z

LoadError: [91mpromotion of types Float64 and Infinitesimal{Int64} failed to change any arguments[39m

In [8]:
function zero(x::Infinitesimal{T}) where T
    Infinitesimal(zero(x.x), zero(x.dx))
end

zero (generic function with 23 methods)

In [9]:
function promote_rule(::Type{Infinitesimal{T}}, ::Type{Infinitesimal{S}}) where {T,S}
    Infinitesimal{promote_type(T,S)}
end
function promote_rule(::Type{Infinitesimal{T}}, ::Type{S}) where {T, S <: Number}
    Infinitesimal{promote_type(T,S)}
end
function promote_rule(::Type{T}, ::Type{Infinitesimal{S}}) where {T <: Number, S}
    Infinitesimal{promote_type(T,S)}
end
function promote_rule(::Type{S}, ::Type{Infinitesimal{T}}) where {S <: AbstractIrrational, T}
    Infinitesimal{promote_type(S,T)}
end

promote_rule (generic function with 126 methods)

In [10]:
function extract_derivative(xdx::Infinitesimal)
    return xdx.dx
end
function extract_derivative(xs::Array)
    [extract_derivative(x) for x in xs]
end
function extract_derivative(xs::Tuple)
    convert(Tuple, [extract_derivative(x) for x in xs])
end

extract_derivative (generic function with 3 methods)

# Differential operator

In [11]:
function D(f)
    function df(x)
        xdx = Infinitesimal(x, one(x))
        result = f(xdx)
        return extract_derivative(result)
    end
    df
end
function D(i::Integer, f)
    function df(xs...)
        xarr = [(j != i ? x : Infinitesimal(x, one(x))) for (j,x) in enumerate(xs)]
        result = f(xarr...)
        return extract_derivative(result)
    end
    df
end

D (generic function with 2 methods)

In [12]:
function +(x::Infinitesimal, y::Infinitesimal)
    Infinitesimal(x.x+y.x, x.dx+y.dx)
end
function -(x::Infinitesimal, y::Infinitesimal)
    Infinitesimal(x.x-y.x, x.dx-y.dx)
end
function -(x::Infinitesimal)
    Infinitesimal(-x.x, -x.dx)
end
function *(x::Infinitesimal, y::Infinitesimal)
    Infinitesimal(x.x*y.x, x.x*y.dx + x.dx*y.x)
end
function /(x::Infinitesimal, y::Infinitesimal)
    Infinitesimal(x.x/y.x, x.dx/y.x - x.x*y.dx/(y.x*y.x))
end
function /(x::Array, y::Number)
    return [i / y for i in x]
end
function ^(a::Infinitesimal, x::Infinitesimal)
    Infinitesimal(a.x^x.x,a.x^x.x*log(a.x)*x.dx)
end

^ (generic function with 67 methods)

In [13]:
function sqrt(x::Infinitesimal)
    Infinitesimal(sqrt(x.x), x.dx/(2*sqrt(x.x)))
end

sqrt (generic function with 21 methods)

In [14]:
function exp(x::Infinitesimal)
    return Infinitesimal(exp(x.x), exp(x.x)*x.dx)
end
function exp(xs::Array)
    return [exp(x) for x in xs]
end
function sin(x::Infinitesimal)
    return Infinitesimal(sin(x.x), cos(x.x)*x.dx)
end
function cos(x::Infinitesimal)
    return Infinitesimal(cos(x.x), -sin(x.x)*x.dx)
end

cos (generic function with 14 methods)

In [15]:
function max(x::Infinitesimal)
    return Infinitesimal(max(0,x.x), x.x < 0 ? 0 : 1 * x.dx)
end

max (generic function with 13 methods)

In [16]:
function iterate(iter::Infinitesimal, state=1)
    if state > length(iter.val)
        return nothing
    end
    return (iter[state],state+1)
end

iterate (generic function with 212 methods)

In [17]:
@test begin
    x = randn()
    isapprox(D(exp)(x), exp(x))
end

Test Passed

In [18]:
function f(x)
    return exp(-x/pi)*sin(2.0*pi*sqrt(2)*x)
end
fprime = D(f)

(::var"#df#1"{typeof(f)}) (generic function with 1 method)

In [19]:
fprime(3)

0.03569811122195761

In [20]:
function f(x)
#     return exp(-x/pi)*sin(2.0*pi*sqrt(2)*x)
    return sin(x*x)
end
fprime = D(f)

(::var"#df#1"{typeof(f)}) (generic function with 1 method)

In [21]:
D(2)

(::var"#df#1"{Int64}) (generic function with 1 method)

In [22]:
function linear(x, m, b)
    return m*x+b
end


linear (generic function with 1 method)

In [23]:
dldx = D(1, linear)

(::var"#df#3"{Int64,typeof(linear)}) (generic function with 1 method)

In [24]:
dldx(1,2,3)

2

In [25]:
function relu(x)
    return max(0,x)
end

relu (generic function with 1 method)

In [26]:
function f(x)
    return x+relu(x*x)
end

f (generic function with 1 method)

In [27]:
d= D(f)

(::var"#df#1"{typeof(f)}) (generic function with 1 method)

In [28]:
d(-2)

LoadError: [91mMethodError: no method matching isless(::Infinitesimal{Int64}, ::Int64)[39m
[91m[0mClosest candidates are:[39m
[91m[0m  isless([91m::Missing[39m, ::Any) at missing.jl:87[39m
[91m[0m  isless([91m::AbstractFloat[39m, ::Real) at operators.jl:167[39m
[91m[0m  isless([91m::Real[39m, ::Real) at operators.jl:355[39m
[91m[0m  ...[39m

In [29]:
function softmax(vector::Array)
    e = exp(vector)
    return e / sum(e)
end

softmax (generic function with 1 method)

In [30]:
d=D(softmax)

(::var"#df#1"{typeof(softmax)}) (generic function with 1 method)

In [31]:
d(2, [1,2,3])

LoadError: [91mMethodError: no method matching (::var"#df#1"{typeof(softmax)})(::Int64, ::Array{Int64,1})[39m
[91m[0mClosest candidates are:[39m
[91m[0m  df(::Any) at In[11]:2[39m

In [32]:
function ftest(x)
    return 2^x
end

ftest (generic function with 1 method)

In [33]:
d=D(ftest)

(::var"#df#1"{typeof(ftest)}) (generic function with 1 method)

In [34]:
d(2)

LoadError: [91mMethodError: no method matching Infinitesimal(::Int64, ::Float64)[39m
[91m[0mClosest candidates are:[39m
[91m[0m  Infinitesimal(::T, [91m::T[39m) where T<:Number at In[3]:2[39m
[91m[0m  Infinitesimal(::T) where T<:Number at boot.jl:716[39m

In [35]:
typeof(x)

LoadError: [91mUndefVarError: x not defined[39m

In [36]:
function szmurlo(x,x2,x3)
    return sin(x)^4+cos(sin(x2))*x3
end

szmurlo (generic function with 1 method)

In [37]:
d = D(1, szmurlo)

(::var"#df#3"{Int64,typeof(szmurlo)}) (generic function with 1 method)

In [41]:
d(1,1,1)

1.287698674479646